In [1]:
import pandas as pd

from src.data_utils.loader import DataLoader
from src.models.config import ModelConfig
from src.models.trainer import AssetPricingTrainer

c:\Users\rcorr\anaconda3\envs\asset_pricing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiments = [
    ModelConfig.OLS_Huber(
        name="OLS+H", 
        feature_set="all"
    ),
    
    ModelConfig.OLS_Huber(
        name="OLS-3+H", 
        feature_set="ff3"
    ),
    
    ModelConfig.ElasticNet_Huber(
        name="ENET+H", 
        feature_set="all", 
        trials=30
    )
]

In [ ]:
DB_PATH = "data/processed/lakehouse.sqlite"
# ff5_features = ['mvel1', 'bm', 'mom12m', 'mom1m', 'retvol'] # Not fama french 5, i think

loader = DataLoader(DB_PATH)
df = loader.load_panel_data(
    # feature_cols = ff5_features, 
    target_col = 'target_ret_excess'
)

trainer = AssetPricingTrainer(df)

[2026-02-05 15:50:54] [INFO] [DB_Loader] STREAMING DATA, ALL COLUMNS...


In [ ]:
# results = []
# for exp in experiments:
#     metrics = trainer.run_experiment(exp)
#     metrics['model'] = exp.name
#     results.append(metrics)

In [ ]:
# res_df = pd.DataFrame(results)
# print("BASELINE LEADERBOARD")
# print(res_df[['model', 'r2_oos', 'rmse']])